In [266]:
from pubg_util import mysql, load_schema, notifier
from sphynx import sphynx, NODE_SMALL, NODE_MEDIUM, NODE_LARGE
from pyspark.sql.functions import *
import pandas as pd
import pickle

spark = sphynx.get_spark(executor_count=4, app_name='pdu_syp', node_spec=NODE_LARGE)

Spark cluster not assigned. creating a new one...
Node spec: 4 executors with 48G RAM each
Job Port 4049 is assigned for requested cluster
Waiting for Spark master to be available...
Spark master launched!
Creating new Spark session, name: pdu_syp...
Waiting for all executors ready...
All executors connected!
Complete! elapsed time: 00:00:28


In [265]:
sphynx.stop()

Stopping Spark session...
Destroying Spark cluster...
Done!
You can safely ignore the warning message if there is any.


In [19]:
june_user = load_data_mart(start_date="2021-06-02", end_date="2021-10-02", device="pc", table_name="user_master").where(col("lastlogindate") >= col("date"))

In [20]:
june_user_pd = june_user.groupBy("date").agg(countDistinct("accountid").alias("au")).toPandas()

In [22]:
june_user_pd.to_csv("./data/user1.csv", index=False)

In [24]:
user2 = load_data_mart(start_date="2021-10-03", end_date="2021-12-11", device="pc", table_name="user_master").where(col("lastlogindate") >= col("date"))

In [25]:
user2_pd = user2.groupBy("date").agg(countDistinct("accountid").alias("au")).toPandas()

In [26]:
user2_pd.to_csv("./data/user2.csv", index=False)

In [27]:
user = pd.concat([june_user_pd, user2_pd])

In [31]:
user = user.sort_values("date")

In [33]:
user.to_csv("./data/au.csv", index=False)

In [84]:
user.head()

,date,au
44,2021-06-02,1229436
115,2021-06-03,1548998
13,2021-06-04,1593057
104,2021-06-05,1715203
83,2021-06-06,1726872


In [151]:
# user pandas dataframe to spark dataframe
user_ = spark.createDataFrame(user)

- 2021-06-02 ~ 2021-10-02
- 2021-08-11 ~ 2021-12-11
- 2021-10-06 ~ 2021-12-06

In [287]:
device = 'pc' 
env = 'live' 
log_name = 'ItemEquipped'
start_date = '2021-06-02'
end_date = '2021-12-06' 
item_equipped = load_schema.lobby(spark, device, env, log_name, start_date, end_date)

In [288]:
with open("./data/partId_list.pickle", "rb") as f:
    partId_list = pickle.load(f)

In [289]:
item_equipped = item_equipped.where(col("PartId").isin(partId_list)).select("date", "AccountId", "ItemId", "PartId").distinct()

In [126]:
item_equipped.printSchema()

root
 |-- date: string (nullable = true)
 |-- AccountId: string (nullable = true)
 |-- ItemId: string (nullable = true)
 |-- PartId: string (nullable = true)



In [73]:
mysql.insert_table(item_equipped, "labs", "daily_item_equipped")

In [74]:
wsus_id_list = ["itemdesc.12012001", "itemdesc.12012002", "itemdesc.12012003"]

In [75]:
wsus_equipped = item_equipped.where(col("ItemId").isin(wsus_id_list)).groupBy("date", "ItemId").agg(countDistinct("AccountId").alias("equipped_cnt")).toPandas()

In [147]:
wsus6_user_list = item_equipped.where((col("ItemId").isin("itemdesc.12012001"))).select("AccountId").distinct().toPandas()["AccountId"].tolist()

In [148]:
wsus8_user_list = item_equipped.where((col("ItemId").isin("itemdesc.12012002"))).select("AccountId").distinct().toPandas()["AccountId"].tolist()

In [149]:
wsus10_user_list = item_equipped.where((col("ItemId").isin("itemdesc.12012003"))).select("AccountId").distinct().toPandas()["AccountId"].tolist()

In [155]:
user_ = user_.select(col("date").alias("date_"), col("au"))

In [157]:
item_equipped = item_equipped.join(user_, item_equipped.date == user_.date_, how="left")

In [160]:
item_equipped.printSchema()

root
 |-- date: string (nullable = true)
 |-- AccountId: string (nullable = true)
 |-- ItemId: string (nullable = true)
 |-- PartId: string (nullable = true)
 |-- date_: string (nullable = true)
 |-- au: long (nullable = true)



In [184]:
items = mysql.read_table(spark, "metainfo", "item_meta_sales").groupBy("product_id", "product_name").agg(count("*")).select("product_id", "product_name")
item_equipped = item_equipped.join(items, item_equipped.ItemId == items.product_id, how="left")

In [290]:
wsus6_user = item_equipped.where(col("ItemId") == "itemdesc.12012001").groupBy("AccountId").agg(min(col("date")).alias("min_date")).select(col("AccountId").alias("account_id"), col("min_date"))
wsus8_user = item_equipped.where(col("ItemId") == "itemdesc.12012002").groupBy("AccountId").agg(min(col("date")).alias("min_date")).select(col("AccountId").alias("account_id"), col("min_date"))
wsus10_user = item_equipped.where(col("ItemId") == "itemdesc.12012003").groupBy("AccountId").agg(min(col("date")).alias("min_date")).select(col("AccountId").alias("account_id"), col("min_date"))

In [186]:
wsus6_item_equipped = item_equipped.join(wsus6_user, (item_equipped.AccountId == wsus6_user.account_id) & (item_equipped.date >= wsus6_user.min_date))
wsus8_item_equipped = item_equipped.join(wsus8_user, (item_equipped.AccountId == wsus8_user.account_id) & (item_equipped.date >= wsus8_user.min_date))
wsus10_item_equipped = item_equipped.join(wsus10_user, (item_equipped.AccountId == wsus10_user.account_id) & (item_equipped.date >= wsus10_user.min_date))

In [158]:
# wsus6_item_equipped = item_equipped.where(col("AccountId").isin(wsus6_user_list))
# wsus8_item_equipped = item_equipped.where((col("AccountId").isin(wsus8_user_list)) & (col("date") >= "2021-08-11"))
# wsus10_item_equipped = item_equipped.where((col("AccountId").isin(wsus10_user_list)) & (col("date") >= "2021-10-06"))

In [188]:
# mysql.drop_table("labs", "wsus6_item_equipped")
# mysql.drop_table("labs", "wsus8_item_equipped")
# mysql.drop_table("labs", "wsus10_item_equipped")

In [189]:
mysql.insert_table(wsus6_item_equipped, "labs", "wsus6_item_equipped")
mysql.insert_table(wsus8_item_equipped, "labs", "wsus8_item_equipped")
mysql.insert_table(wsus10_item_equipped, "labs", "wsus10_item_equipped")

In [200]:
item_equipped.where((col("date") == "2021-11-04") & (col("ItemId") == "itemdesc.12010509")).show(truncate=False)

+----------+----------------------------------------+-----------------+-------------+----------+-------+----------+------------+
|date      |AccountId                               |ItemId           |PartId       |date_     |au     |product_id|product_name|
+----------+----------------------------------------+-----------------+-------------+----------+-------+----------+------------+
|2021-11-04|account.f6230738af894bbb87eba342abaf38a1|itemdesc.12010509|partdesc.m762|2021-11-04|1164014|null      |null        |
|2021-11-04|account.c7e928523bb44ccb8c6676281ce1d2a3|itemdesc.12010509|partdesc.m762|2021-11-04|1164014|null      |null        |
+----------+----------------------------------------+-----------------+-------------+----------+-------+----------+------------+



In [191]:
wsus6_item_equipped.printSchema()

root
 |-- date: string (nullable = true)
 |-- AccountId: string (nullable = true)
 |-- ItemId: string (nullable = true)
 |-- PartId: string (nullable = true)
 |-- date_: string (nullable = true)
 |-- au: long (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_name: string (nullable = true)
 |-- account_id: string (nullable = true)
 |-- min_date: string (nullable = true)



In [194]:
wsus6_tmp = wsus6_item_equipped.groupBy("date", "ItemId").agg(countDistinct("AccountId").alias("user_cnt"), avg("au").alias("au")).withColumn("eur", col("user_cnt")/col("au")).toPandas()
wsus8_tmp = wsus8_item_equipped.groupBy("date", f"ItemId").agg(countDistinct("AccountId").alias("user_cnt"), avg("au").alias("au")).withColumn("eur", col("user_cnt")/col("au")).toPandas()
wsus10_tmp = wsus10_item_equipped.groupBy("date", "ItemId").agg(countDistinct("AccountId").alias("user_cnt"), avg("au").alias("au")).withColumn("eur", col("user_cnt")/col("au")).toPandas()

In [312]:
items = mysql.read_table(spark, "metainfo", "meta_vc_sales_items").groupBy("product_id", "product_name").agg(count("*")).select("product_id", "product_name").toPandas()

In [323]:
pd.merge(wsus6_tmp, items, left_on="ItemId", right_on="product_id", how="left").to_csv("./data/wsus6_tmp.csv", index=False)
pd.merge(wsus8_tmp, items, left_on="ItemId", right_on="product_id", how="left").to_csv("./data/wsus8_tmp.csv", index=False)
pd.merge(wsus10_tmp, items, left_on="ItemId", right_on="product_id", how="left").to_csv("./data/wsus10_tmp.csv", index=False)

In [76]:
wsus_equipped.to_csv("./data/wsus_equipped.csv", index=False)

In [81]:
wsus_equipped.dtypes

date            object
ItemId          object
equipped_cnt     int64
dtype: object

In [87]:
user.date = user.date.astype(str)
wsus_equipped.date = wsus_equipped.date.astype(str)

In [90]:
pd.merge(wsus_equipped, user, on="date", how="left").to_csv("./data/wsus_rate.csv", index=False)

In [98]:
item_equipped = mysql.read_table(spark, "labs", "daily_item_equipped")

In [99]:
item_equipped.printSchema()

root
 |-- date: string (nullable = true)
 |-- AccountId: string (nullable = true)
 |-- ItemId: string (nullable = true)
 |-- PartId: string (nullable = true)



In [128]:
# mysql.drop_table("labs", "daily_wsus_item_equipped")

당일 성장형 무기를 사용한 사람들의 사용 무기

In [129]:
wsus_item_equipped = item_equipped.where(col("ItemId").isin(wsus_id_list)).select("date", "AccountId").distinct()
mysql.insert_table(wsus_item_equipped, "labs", "daily_wsus_item_equipped_user")

In [135]:
wsus_item_equipped2 = wsus_item_equipped.select(col("date").alias("date2"), col("AccountId").alias("AccountId2"))

In [136]:
wsus_df = item_equipped.join(wsus_item_equipped2, (item_equipped.date == wsus_item_equipped2.date2) & (item_equipped.AccountId == wsus_item_equipped2.AccountId2))

In [137]:
wsus_df.printSchema()

root
 |-- date: string (nullable = true)
 |-- AccountId: string (nullable = true)
 |-- ItemId: string (nullable = true)
 |-- PartId: string (nullable = true)
 |-- date2: string (nullable = true)
 |-- AccountId2: string (nullable = true)



In [138]:
wsus_pd = wsus_df.toPandas()

In [139]:
wsus_pd.to_csv("./data/wsus_pd.csv", index=False)

In [101]:
equipped = item_equipped.groupby("date", "ItemId").agg(countDistinct("AccountId").alias("user_cnt")).toPandas()

In [103]:
equipped.to_csv("./data/equipped.csv", index=False)

In [107]:
pd.merge(equipped, user, on="date", how="left").to_csv("./data/equipped_rate.csv", index=False)

In [111]:
items = items.toPandas()

In [113]:
tmp = pd.merge(equipped, user, on="date", how="left")
equipped = pd.merge(tmp, items, left_on="ItemId", right_on="product_id", how="left")
equipped.to_csv("./data/equipped_rate_with_name.csv", index=False)

In [116]:
equipped.sort_values("user_cnt", ascending=False).head(10)

,date,ItemId,user_cnt,au,product_id,product_name
64287,2021-09-08,itemdesc.12010481,126589,1219804,NaN,NaN
42552,2021-09-08,itemdesc.12010471,103857,1219804,NaN,NaN
16968,2021-07-07,itemdesc.12010447,84282,1372117,NaN,NaN
6217,2021-11-12,itemdesc.12010509,67652,1243962,NaN,NaN
84748,2021-11-12,itemdesc.12010508,56913,1243962,NaN,NaN
41196,2021-09-17,itemdesc.12010475,44289,1447139,itemdesc.12010475,White Rabbit - Pan
80337,2021-06-23,itemdesc.12010439,42016,1432555,itemdesc.12010439,CNH's SLR
37538,2021-07-08,itemdesc.12010447,40488,1652825,NaN,NaN
38424,2021-07-09,itemdesc.12010442,33820,1688244,NaN,NaN
75837,2021-09-17,itemdesc.12010477,33383,1447139,itemdesc.12010477,Bunny Aeronautics - M416


- 12010468: PCS5 Beryl
- 12010467: PCS5 SLR


In [291]:
au = load_data_mart(start_date="2021-09-08", end_date="2021-09-21", device="pc", table_name="user_master").where(col("lastlogindate") >= col("date")).select("date", "accountid")

In [292]:

wsus6_active = au.join(wsus6_user, (au.date >= wsus6_user.min_date) & (au.accountid == wsus6_user.account_id), how="left")

In [293]:
wsus8_active = au.join(wsus8_user, (au.date >= wsus8_user.min_date) & (au.accountid == wsus8_user.account_id), how="left")

In [243]:
item_equipped.printSchema()

root
 |-- date: string (nullable = true)
 |-- AccountId: string (nullable = true)
 |-- ItemId: string (nullable = true)
 |-- PartId: string (nullable = true)
 |-- date_: string (nullable = true)
 |-- au: long (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_name: string (nullable = true)



In [233]:
wsus6_active.printSchema()
# account_id가 null이면 성장형 무기 사용 x

root
 |-- date: date (nullable = true)
 |-- accountid: string (nullable = true)
 |-- account_id: string (nullable = true)
 |-- min_date: string (nullable = true)



In [247]:
wsus6_active_cnt = wsus6_active.withColumn("is_wsus", when(col("account_id").isNull(), 0).otherwise(1)).groupBy("date", "is_wsus").agg(count("*").alias("user_cnt")).toPandas()

In [251]:
wsus8_active_cnt = wsus8_active.withColumn("is_wsus", when(col("account_id").isNull(), 0).otherwise(1)).groupBy("date", "is_wsus").agg(count("*").alias("user_cnt")).toPandas()

In [252]:
# wsus6_active_cnt.to_csv("./data/wsus6_active_cnt.csv", index=False)
# wsus8_active_cnt.to_csv("./data/wsus8_active_cnt.csv", index=False)

In [302]:
a = wsus6_active.select(col("date").alias("date_a"), col("accountid").alias("AccountId_active"), col("account_id"))
b = item_equipped.where((col("date") >= "2021-09-08") & (col("date") <= "2021-09-21") & (col("ItemId").isin(["itemdesc.12010467", "itemdesc.12010468"]))).select("date", "AccountId", "ItemId")
wsus6_active_equip = b.join(a, (b.AccountId == a.AccountId_active) & (b.date == a.date_a), how="right")

In [305]:
a = wsus8_active.select(col("date").alias("date_a"), col("accountid").alias("AccountId_active"), col("account_id"))
b = item_equipped.where((col("date") >= "2021-09-08") & (col("date") <= "2021-09-21") & (col("ItemId").isin(["itemdesc.12010467", "itemdesc.12010468"]))).select("date", "AccountId", "ItemId")
wsus8_active_equip = b.join(a, (b.AccountId == a.AccountId_active) & (b.date == a.date_a), how="right")

In [303]:
pcs_wsus6 = wsus6_active_equip.withColumn("group", when((col("account_id").isNull()) & (col("AccountId").isNotNull()), 'no_wsus').when((col("account_id").isNotNull()) & (col("AccountId").isNotNull()), 'wsus').otherwise("none")).groupBy("date", "ItemId", "group").agg(count("*").alias("user_cnt")).toPandas()


In [306]:
pcs_wsus8 = wsus8_active_equip.withColumn("group", when((col("account_id").isNull()) & (col("AccountId").isNotNull()), 'no_wsus').when((col("account_id").isNotNull()) & (col("AccountId").isNotNull()), 'wsus').otherwise("none")).groupBy("date", "ItemId", "group").agg(count("*").alias("user_cnt")).toPandas()


In [308]:
pcs_wsus6.to_csv("./data/pcs_wsus6.csv", index=False)
pcs_wsus8.to_csv("./data/pcs_wsus8.csv", index=False)

In [333]:
pcs_wsus6.head()

,date,ItemId,group,user_cnt
0,2021-09-19,itemdesc.12010468,wsus,612
1,2021-09-08,itemdesc.12010467,no_wsus,21134
2,2021-09-20,itemdesc.12010467,wsus,391
3,2021-09-15,itemdesc.12010468,no_wsus,3559
4,2021-09-15,itemdesc.12010468,wsus,664


In [334]:
wsus6_active_cnt.loc[wsus6_active_cnt[wsus6_active_cnt.is_wsus == 1].index, "is_wsus"] = "wsus"

In [336]:
wsus6_active_cnt.loc[wsus6_active_cnt[wsus6_active_cnt.is_wsus == 0].index, "is_wsus"] = "no_wsus"

In [348]:
wsus8_active_cnt.loc[wsus8_active_cnt[wsus8_active_cnt.is_wsus == 1].index, "is_wsus"] = "wsus"
wsus8_active_cnt.loc[wsus8_active_cnt[wsus8_active_cnt.is_wsus == 0].index, "is_wsus"] = "no_wsus"

In [345]:
# wsus6_active_cnt.date = wsus6_active_cnt.date.astype(str)
# pcs_wsus6.date = pcs_wsus6.date.astype(str)
# wsus8_active_cnt.date = wsus8_active_cnt.date.astype(str)
# pcs_wsus8.date = pcs_wsus8.date.astype(str)

In [350]:
pcs_wsus6_df = pd.merge(pcs_wsus6, wsus6_active_cnt, left_on=["date", "group"], right_on=["date", "is_wsus"])
pcs_wsus8_df = pd.merge(pcs_wsus8, wsus8_active_cnt, left_on=["date", "group"], right_on=["date", "is_wsus"])

In [352]:
pcs_wsus6_df["rate"] = pcs_wsus6_df.user_cnt_x/pcs_wsus6_df.user_cnt_y
pcs_wsus8_df["rate"] = pcs_wsus8_df.user_cnt_x/pcs_wsus8_df.user_cnt_y

In [355]:
pd.merge(pcs_wsus6_df, items, left_on="ItemId", right_on="product_id").to_csv("./data/pcs_wsus6_df.csv", index=False)
pd.merge(pcs_wsus8_df, items, left_on="ItemId", right_on="product_id").to_csv("./data/pcs_wsus8_df.csv", index=False)

In [356]:
pcs_wsus6_df = pd.read_csv("./data/pcs_wsus6_df.csv")
pcs_wsus8_df = pd.read_csv("./data/pcs_wsus8_df.csv")

In [363]:
pcs_wsus6_df.groupby(["group", "product_name"], as_index=False)["rate"].mean().sort_values("group")

,group,product_name,rate
0,no_wsus,PCS5 Shrapnel Scrawl - Beryl M762,0.004676
1,no_wsus,PCS5 Shrapnel Scrawl - SLR,0.003731
2,wsus,PCS5 Shrapnel Scrawl - Beryl M762,0.014535
3,wsus,PCS5 Shrapnel Scrawl - SLR,0.014127


In [364]:
pcs_wsus8_df.groupby(["group", "product_name"], as_index=False)["rate"].mean().sort_values("group")

,group,product_name,rate
0,no_wsus,PCS5 Shrapnel Scrawl - Beryl M762,0.004826
1,no_wsus,PCS5 Shrapnel Scrawl - SLR,0.004051
2,wsus,PCS5 Shrapnel Scrawl - Beryl M762,0.015075
3,wsus,PCS5 Shrapnel Scrawl - SLR,0.010951
